In [1]:
# -*- coding: utf-8 -*-
import dataiku
import pandas as pd, numpy as np
from dataiku import pandasutils as pdu
from gensim.models import word2vec
import itertools
from itertools import product

/Users/mmiyazaki/dataiku/Design/DATA_DIR/code-envs/python/japan-nlp/lib/python3.6/site-packages/scipy/sparse/sparsetools.py:21: DeprecationWarning: `scipy.sparse.sparsetools` is deprecated!
scipy.sparse.sparsetools is a private module for scipy.sparse, and should not be used.
  _deprecated()


In [2]:
def cos_sim(v1, v2):
    return np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))

In [3]:
# Read recipe inputs
model_folder_path = dataiku.Folder("m9JZdV7b").get_path()
model_path = model_folder_path + "/word2vec_ramen_model.model"
ramen_model = word2vec.Word2Vec.load(model_path)

In [4]:
reviews_TF_IDF = dataiku.Dataset("reviews_TF_IDF")
df = reviews_TF_IDF.get_dataframe()

In [5]:
df.head()

,store_name,address,ward,score,review_cnt,review,texts_tfidf_sorted_top20,id
0,手打式超多加水麺 ののくら,東京都葛飾区亀有3-11-11マーベラス大協ビル 1F,東京都内,3.98,479,2020年、ラーメン103杯目(^^)本日、朝イチで営業を済ませ、次の打ち合わせまで時間が空...,"['うま味', 'えのき', 'パイン', '町田', '仲見世商店街', '町田駅', '...",ID-000001
1,Homemade Ramen 麦苗,東京都品川区南大井6-11-10,東京都内,3.97,731,2020/08/03(月)5度目の訪問！現在は蜜を避けるために、事前記帳(予約)式になってい...,"['afuri', '恵比寿', '柚子', '中村屋', '利山', '中村', '天然水...",ID-000002
2,麺尊 RAGE,東京都杉並区松庵3-37-22レンツェン松庵 1F,東京都内,3.96,723,5年連続で「ミシュランガイド東京」のビブグルマンに掲載され、「The Tabelog Awa...,"['江戸川橋', '油そば', '中華そば', '江戸川橋駅', '神楽', 'そば', '...",ID-000003
3,らぁ麺や 嶋,東京都渋谷区本町3-41-12,東京都内,3.95,177,今年、西新宿五丁目に新しいラーメン屋が出来た。まだ1年も経っていないだろうにすでに期待度が非...,"['記帳', 'いりこ', '黒毛和牛', 'ウェイティング', '火山', 'ボード', ...",ID-000004
4,宍道湖しじみ中華蕎麦 琥珀,東京都大田区西六郷2-1-3,東京都内,3.95,296,本日は貴重な平日のお休みだった為、普段は中々足を運ぶ事の出来ない蒲田方面へ。11時40分に蒲...,"['黒毛和牛', 'soba', '代々木上原', 'トリュフ', '青森シャモロック', ...",ID-000005


In [6]:
tripAdvisor_path = dataiku.Folder("FXMfF0DU").get_path() + "/TripAdvisor_top_TFIDF_words.txt"
f = open(tripAdvisor_path,'r',encoding="utf-8")

In [7]:
words_ta = []
for i,data in enumerate(f):
    word = data.replace("'",'').replace('[','').replace(']','').replace(' ','').replace('\n','')
    words_ta.append(word)

In [8]:
words_ta

['ロンドン',
 'ハマチ',
 'うなぎ',
 '疑い',
 'メインディッシュ',
 '宝石',
 'すし',
 'お刺身',
 '効率的',
 '黒ごま',
 'リラックス',
 '共有',
 '汁物',
 'サーモン',
 'いたこと',
 '推薦',
 'しみ',
 '前菜',
 '盛り合わせ',
 '高品質']

In [9]:
word_ta_vectors = [ramen_model.wv[w] for w in words_ta]

In [11]:
word_list = []
arr = df['texts_tfidf_sorted_top20'].values
for a in arr:
    l = a.replace("'", "").replace("[", "").replace("]", "").replace(" ", "").split(",")
    word_list.append(l)

In [12]:
word_list

[['うま味',
  'えのき',
  'パイン',
  '町田',
  '仲見世商店街',
  '町田駅',
  'パパパパパイン',
  'ノー',
  '乾燥',
  'パイナップル',
  'タイル',
  'ホタテ',
  '椎茸',
  '帆立',
  '塩そ',
  'そば',
  '辛味',
  'ホンビノス貝',
  'トリプル',
  '鮮魚'],
 ['afuri',
  '恵比寿',
  '柚子',
  '中村屋',
  '利山',
  '中村',
  '天然水',
  '栄利',
  '恵比寿駅',
  'zundbar',
  '天空',
  '辣湯',
  'タッチパネル',
  '神奈川県',
  '海老名',
  '従業員',
  'こんにゃく',
  'まろ',
  '改定',
  '水菜'],
 ['江戸川橋',
  '油そば',
  '中華そば',
  '江戸川橋駅',
  '神楽',
  'そば',
  'つけ',
  'ドリル',
  'しな',
  '煮干し',
  'マン',
  '神楽坂駅',
  '中盛',
  '神楽坂',
  '特製',
  'か中',
  'ナルト',
  '粘り気',
  'プルプル',
  'zma'],
 ['記帳',
  'いりこ',
  '黒毛和牛',
  'ウェイティング',
  '火山',
  'ボード',
  'spf',
  'homemaderamen',
  '予約',
  '洗車',
  '大森',
  '記名',
  '空き',
  '記入',
  '伊吹',
  '雲呑',
  'ワンタン',
  '往訪',
  'ワンタン麺',
  'オリーブ'],
 ['黒毛和牛',
  'soba',
  '代々木上原',
  'トリュフ',
  '青森シャモロック',
  'ポルチーニ',
  '胡桃',
  'sukiyaki',
  'wagyu',
  'フォンド',
  '移転',
  '代々木上原駅',
  '北京ダック',
  '天草大王',
  '九条葱',
  '巣鴨',
  'japanesesobanoodles',
  'ミシュラン',
  'しゃぶしゃぶ',
  'レバーペースト'],
 ['kaeru',
  '玉ねぎ',
  '二郎系',
  '

In [0]:
word_list.append(words_ta)

In [0]:
uniq_words = list(set(itertools.chain.from_iterable(word_list)))
scores = {}
for word1, word2 in product(uniq_words, repeat=2):
    # print(word1, word2)
    scores[(word1, word2)] =  cos_sim(ramen_model.wv[word1], ramen_model.wv[word2])

In [0]:
scores

In [0]:
df

In [0]:
avg_avg_scores = []
for word_2 in words_ta:
    avg_scores = []
    for review in df["texts_tfidf_sorted_top20"].values:
        review = review.replace("'", "").replace("[", "").replace("]", "").replace(" ", "").split(",")
        word_cross_scores = []
        for word_1 in review:
            score = scores[(word_1, word_2)]
            word_cross_scores.append(score)
        avg_scores.append(np.mean(word_cross_scores))
    avg_avg_scores.append(np.mean(avg_scores))

In [0]:
avg_avg_scores = []
for review in df["texts_tfidf_sorted_top20"].values:
    review = review.replace("'", "").replace("[", "").replace("]", "").replace(" ", "").split(",")
    avg_scores = []
    for word_1 in review:
        word_cross_scores = []
        for word_2 in words_ta:
            score = scores[(word_1, word_2)]
            word_cross_scores.append(score)
        avg_scores.append(np.mean(word_cross_scores))
    avg_avg_scores.append(np.mean(avg_scores))

In [0]:
len(avg_avg_scores)

In [0]:
df['similarity_score'] = avg_avg_scores

In [0]:
recommendation = dataiku.Dataset("recommendation")
recommendation.write_with_schema(df)